# Cuaderno 29b: Corte de capacidad mínima en un grafo

Implementamos en este cuaderno las funciones para encontrar cortes de capacidad mínima, en grafos dirigidos y no dirigidos, que son requeridas por el algoritmo de planos cortantes para el problema del agente viajero descrito en el Cuaderno 27.


## Grafos no dirigidos

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados 
* un grafo no dirigido $G = (V, E)$, y
* un vector de capacidades $u \in \ZZ^E$ asociado a las aristas de $E$,

el problema del *corte de capacidad mínima* consiste en encontrar un conjunto de nodos $W \subset V$, con $\emptyset \neq W \neq V$, tal que la capacidad del corte:
$$
u(\delta(W)) := \sum_{ij \in \delta(W)} \!\!\! u_{ij}
$$
sea mínima. 

El algoritmo de **Stoer-Wagner** permite resolver este algoritmo de una manera eficiente. El algoritmo consiste en repetir $n-1$ veces la siguiente rutina *MinimumCutPhase*:
1. Seleccionar un nodo arbitrario $a \in V$ y definir $A:= \{ a \}$.
2. Mientras $V \neq A$, hacer:
  1. Seleccionar $i \in V \setminus A$ tal que las aristas incidentes a $i$ que tienen el otro extremo en $A$ tengan capacidad total máxima.
  2. $A := A \cup \{ i \}$
3. Sean $v$ y $w$ el penúltimo y último nodos añadidos a $A$, respectivamente. Definir $W:=V \setminus \{w\}$ y asociado a él definir el *corte de la fase* $\delta(W)$.  
4. Contraer los nodos $v$ y $w$ en un *supernodo* $\{w, v\}$. Cada arista incidente a cualquiera de estos nodos es reemplazada por una arista de igual capacidad incidente al supernodo. Si se producen aristas paralelas entre un mismo par de nodos, las mismas son reemplazadas por una única arista con capacidad igual a la suma de las capacidades de todas las aristas paralelas. Como resultado de la contracción, se obtiene un nuevo grafo con un nodo menos.

Puede demostrarse que el corte de la fase obtenido en el paso 3 es el corte de capacidad mínima que además satisface la condición de *separar* a $v$ de $w$. Es decir, un corte $\delta(W)$ tal que $v \in W$ y $w \not\in W$. Por otra parte, luego de contraer $v$ y $w$ en el paso 4, cualquier corte encontrado en una nueva aplicación de *MinimumCutPhase* no podrá separar $v$ y $w$, pues ambos nodos están representados por un único supernodo. Por lo tanto, el problema del corte mínimo puede resolverse al aplicar repetidamente *MinimumCutPhase* hasta llegar a un grafo con 2 nodos y seleccionar luego el corte de fase que tenga la menor capacidad de entre todos los cortes de fase encontrados.

Implementamos a continuación este algoritmo en tres funciones: `corte_minimo` es la función principal, que recibe como parámetros una lista `V` con los nodos del grafo y un diccionario `u` indexado por las aristas del grafo y que almacena sus respectivas capacidades. La función `minimum_cut_phase` implementa recursivamente la rutina *MinimumCutPhase*, la cual es repetida hasta que el grafo tenda 2 nodos. Esta función retorna el menor corte de fase encontrado durante todas las llamadas. Finalmente, la función `merge_graph` implementa la contracción de dos nodos del grafo en un nuevo supernodo.


In [ ]:
import networkx as nx

def corte_minimo(V, u):
    # Parametros:
    # V: lista con los nodos del grafo
    # u: diccionario indexado por las aristas del grafo, con sus capacidades
    # el siguiente diccionario almacena los nodos asociados a cada supernoso
    S = {}
    for i in V:
        S[i]= [i]
    delta, W, S = minimum_cut_phase(V, u, S, V[0])
    # Desagregar supernodos
    W2=[]
    for i in W:
        W2+=S[i]
    W2c = [i for i in V if not i in W2]
    # Retornar capacidad del corte minimo y particionamiento del conjunto de nodos
    return delta, W2, W2c

def minimum_cut_phase(V, u, S, a):
    # si V tiene solamente dos nodos, terminar y retornar capacidad de la unica arista
    if len(V)==2:
        N = [(i,j) for (i,j) in u.keys()]
        i, j = N.pop()
        # print ('V==2', u[i, j], [i], S.copy())
        return u[i, j], [i], S.copy()
    # caso contrario, construir la lista A ejecutando la fase MinimumCut del algoritmo
    G = nx.Graph()
    G.add_weighted_edges_from([(i, j, u[i,j]) for (i,j) in u.keys()], weight='u')
    A = [a]
    # w[i] contiene la suma de las capacidades de las aristas que conectan al nodo i con el conjunto A
    w = {}
    for i in V:
        if i!=a:
            w[i]= 0
    for i in G[a]:
        w[i]= G[a][i]['u']    
    while len(A)!=len(V):
        wmax, i = max([(w[i], i) for i in V if i not in A])
        A.append(i)
        # print i
        # print A
        for j in G[i]:
            # print j
            if j not in A:
                # print (i, j, G[i][j]['u'])
                w[j]= w[j] + G[i][j]['u']
    # A \ A[-1], A[-1] es el corte de capacidad minima que separa A[-1] y A[-2]
    t = A[-1]
    delta1 = sum([G[t][j]['u'] for j in G[t]])
    s = A[-2]
    W1 = A[:-1]
    # print('*1:', delta1, W1, S)
    V2, u2, S2 = merge_graph(V, u, S, s, t)
    # print('*1m:', V2, u2, S2)
    delta2, W2, S2 = minimum_cut_phase(V2, u2, S2, V2[0])
    if delta1 < delta2:
        return delta1, W1, S
    else:
        # print('*2:', delta2, W2, S2)
        return delta2, W2, S2
    
def merge_graph(V, u, S, i, j):
    # crear nuevo conjunto de nodos (eliminar i)
    V2 = [k for k in V if k!=i]
    # crear nuevo diccionario de supernodos
    S2 = {}
    for v in V2:
        S2[v] = S[v][:]
    # agregar S[i] al supernodo indexado como j
    S2[j]+= S[i][:]
    # crear nuevo diccionario de capacidades de aristas
    L = u.keys()
    N1 = [(v, w) for (v, w) in L if i in [v, w] and j not in [v, w]]
    N2 = [(v, w) for (v, w) in L if i not in [v, w]]
    # print('N1:', N1)
    # print('N2:', N2)
    u2 = {}
    for (v, w) in N2:
        u2[v, w]= u[v, w]
    for (v, w) in N1:
        if i==v:
            if (j, w) in u2.keys():
                u2[j, w]+= u[v, w]
            elif (w, j) in u2.keys():
                u2[w, j]+= u[v, w]
            else:
                u2[j, w]= u[v, w]
        else:
            if (j, v) in u2.keys():
                u2[j, v]+= u[v, w]
            elif (v, j) in u2.keys():
                u2[v, j]+= u[v, w]
            else:
                u2[j, v]= u[v, w]
    return V2, u2, S2
    
                

Probemos ahora esta función sobre un grafo no dirigido:

In [ ]:
# lista con los nodos del grafo
V = [1, 2, 3]
# diccionario con las capacidades de las aristas
u = {(1,2): 1, (2,3): 0, (3,1): 1}
# corte de capacidad minima
d, W, Wc = corte_minimo(V, u)
print(d)
print(W)
print(Wc)

## Grafos dirigidos

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados 
* un grafo dirigido $D = (V, A)$, y
* un vector de capacidades $u \in \ZZ^A$ asociado a los arcos de $A$,

el problema del *corte saliente de capacidad mínima* consiste en encontrar un conjunto de nodos $W \subset V$, con $\emptyset \neq W \neq V$, tal que la capacidad saliente del corte:
$$
u(\delta^+(W)) := \!\!\! \sum_{(i,j) \in \delta^+(W)} \!\!\!\!\! u_{ij}
$$
sea mínima. El problema análogo del corte entrante de capacidad mínima consiste en encontrar un $W$ que minimice el valor de $u(\delta^-(W))$.

El algoritmo de **Stoer-Wagner** no puede aplicarse al caso de grafos dirigidos. Una alternativa para calcular un corte saliente (o entrante) de capacidad mínima en un grafo dirigido consiste en resolver $2(n -1)$ problemas de corte-($s$, $t$) de capacidad mínima, con $n = \card{V}$. Cada uno de estos problemas es equivalente a encontrar un flujo-($s$, $t$) de valor máximo.

Formalmente, dados $s, t \in V$ un corte-($s$, $t$) está definido por un conjunto de nodos $W \subset V$ tal que $s \in W$ y $t \not\in W$. Un corte-($s$, $t$) con capacidad saliente $u(\delta^+(W))$ mínima puede obtenerse al calcular un flujo máximo de $s$ a $t$ empleando cualquiera de los algoritmos usuales. 

Para resolver el problema del corte saliente de capacidad mínima, utilizaremos el siguiente esquema:
1. Seleccionar $s \in V$ arbitrario.
2. Para $t \in V \setminus \{ s\}$, hacer:
  1. Calcular un corte-($s$, $t$) de capacidad mínima.
  2. Calcular un corte-($t$, $s$) de capacidad mínima.
3. Retornar el corte de menor capacidad de entre todos los $2(n-1)$ cortes calculados en el paso 2.

Implementamos a continuación este algoritmo, usando la función `minimum_cut`del módulo `networkx` para el cálculo de los cortes-($s$, $t$) de capacidad mínima. 



In [ ]:
import networkx as nx

# corte saliente de capacidad minima
def corte_sal_minimo(V, u):
    # Parametros:
    # V: lista con los nodos del grafo
    # u: diccionario indexado por los arcos del grafo, con sus capacidades
    # se fija un nodo s arbitrario
    s = V[0]
    V2 = [i for i in V if i!=s]
    G = nx.DiGraph()
    G.add_weighted_edges_from([(i, j, u[i,j]) for (i,j) in u.keys()], weight='u')
    # inicializar dmin con la suma de capacidades de todos los arcos
    dmin = sum([u[i,j] for (i, j) in u.keys()])
    # se calculan los cortes minimos (s, t) y (t, s) para todo t != s
    for t in V2: 
        # print('*** s: {}, t:{}'.format(s,t))
        d1, (W1, W1c) = nx.minimum_cut(G, s, t, capacity='u')
        # print(d1, W1)
        d2, (W2, W2c) = nx.minimum_cut(G, t, s, capacity='u')
        # print(d2, W2)
        if d1 < d2 and d1 < dmin:
            dmin, Wmin, Wminc = d1, W1, W1c
        elif d2 <= d1 and d2 < dmin:
            dmin, Wmin, Wminc = d2, W2, W2c
    return dmin, Wmin, Wminc


Probemos el uso de esta función:

In [ ]:
V = [1, 2, 3]
u = {(1, 2): 5, (2, 3): 3, (3, 1): 1}
# corte saliente mínimo en un grafo dirigido
d, W, Wc = corte_sal_minimo(V, u)
print(d)
print(W)
print(Wc)

Apliquemos este algoritmo al grafo dirigido del ejemplo del Cuaderno 27:

In [ ]:
import networkx as nx
import ipycytoscape

# Ejemplo de solución fraccionaria del TSP
# Nodos del grafo
V = range(1,7)

# Arcos (grafo completo)
A = [(i,j) for i in V for j in V if i!=j]

# Valores de las variables en la solución fraccionaria
A1 = [(1,2), (3,1), (6,4), (5,6)]
A2 = [(2,3), (2,5), (4,3), (4,5)]
A3 = [(i,j) for (i,j) in A if (i,j) not in A1+A2 and (j,i) not in A1+A2]
A = A1+A2+A3
x={(i,j) : 1 if (i,j) in A1 else (0.5 if (i,j) in A2 else 0) for (i,j) in A}

D = nx.DiGraph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['etiq']= str(i)
D.add_edges_from(A)
for i,j in A:
    D.edges[i,j]['etiq'] = str(x[i,j]) if x[i,j]>0.1 else ''
    D.edges[i,j]['color'] =  '#0000ff' if x[i,j]>=0.9 else ('#ff007f' if x[i,j]>=0.4 else '#e0e0e0')
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=True)
grafo.set_style([{'selector': 'node', 'style' : {'width': 15, 'height' : 15, 'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
                    {'selector': 'edge', 'style': {'width': 1, 'line-color': 'data(color)', 'font-size': '10px', 'label': 'data(etiq)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

In [ ]:
# corte saliente mínimo 
d, W, Wc = corte_sal_minimo(V, x)
print(W)
print(Wc)
print(d)